# Import Modules


In [1]:
import tensorflow as tf
import keras
from keras.layers import Input,Dense, Dropout, BatchNormalization,Flatten,GlobalAveragePooling2D
from keras.datasets import cifar100 
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras import backend as k 
from keras.utils import np_utils
from __future__ import print_function
import matplotlib.pyplot as plt

import numpy as np
import math
import os
import argparse
import random
from utils import mnist_reader
import sys
import time
from resnet50 import ResNet50
import PIL
from PIL import Image
import glob
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/Users/haohanwang/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


# Cluster Specification 

In [ ]:
# Define input flags to identify the job and task
tf.app.flags.DEFINE_string("job_name", "", "Either 'ps' or 'worker'")
tf.app.flags.DEFINE_integer("task_index", 0, "Index of task within the job")
FLAGS = tf.app.flags.FLAGS

# Create a tensorflow cluster
# Replace localhost with the host names if you are running on multiple hosts
cluster = tf.train.ClusterSpec({"ps": ["localhost:2222"],
                                "worker": [	"localhost:2223",
                                            "localhost:2224",
                                            "localhost:2225"]})
# Start the server
server = tf.train.Server(cluster,
                         job_name=FLAGS.job_name,
                         task_index=FLAGS.task_index)

# Load Data and Data Preprocession

In [2]:
def prepare_input_data(X_train, X_test):
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    return X_train, X_test

def prepare_output_data(y_train, y_test):
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    return y_train, y_test

In [3]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

print("Training images shape: {shape}".format(shape=x_train.shape))
print("Training labels shape: {shape}".format(shape=y_train.shape))

# Shapes of test set
print("Test images shape: {shape}".format(shape=x_test.shape))
print("Test labels shape: {shape}".format(shape=y_test.shape))

Training images shape: (50000, 32, 32, 3)
Training labels shape: (50000, 1)
Test images shape: (10000, 32, 32, 3)
Test labels shape: (10000, 1)


In [6]:
# for i in range(x_train.shape[0]):
#     arr = x_train[i]
#     img = PIL.Image.fromarray(arr, 'RGB')
#     img.save(os.path.join('train_images','image'+str(i)+'.png'))

In [7]:
# for i in range(x_test.shape[0]):
#     arr = x_test[i]
#     img = PIL.Image.fromarray(arr, 'RGB')
#     img.save(os.path.join('test_images','image'+str(i)+'.png'))

In [8]:
# image_list =[]
# i = 0
# for filename in glob.glob('train_images/*.png'):
#     i += 1
#     img = Image.open(filename)
#     new_img = img.resize((256,256))
#     new_img.save(os.path.join('new_train_images', 'image'+str(i)+ '.png'))

In [9]:
# test_image_list =[]
# i = 0
# for filename in glob.glob('test_images/*.png'):
#     i += 1
#     img = Image.open(filename)
#     new_img = img.resize((256,256))
#     new_img.save(os.path.join('new_test_images', 'image'+str(i)+ '.png'))

In [8]:
# num_classes = 100
# result_array = []
# for filename in glob.glob('new_train_images/*.png'):
#     img = Image.open(filename).convert('RGB')
#     arr = np.array(img)
#     result_array.append(arr)
# result_arr = np.asarray(result_array)

In [9]:
# test_array = []
# for filename in glob.glob('new_test_images/*.png'):
#     img = Image.open(filename).convert('RGB')
#     arr = np.array(img)
#     test_array.append(arr)
# test_arr = np.asarray(test_array)

In [4]:
result_arr.tofile('train_image_file', format='%d')
test_arr.tofile('test_image_file', format='%d')

NameError: name 'result_arr' is not defined

In [3]:
x_train = result_arr.reshape([-1,256,256,3])
x_test = test_arr.reshape([-1,256,256,3])
x_train, x_test = prepare_input_data(x_train, x_test)
y_train, y_test = prepare_output_data(y_train, y_test)

NameError: name 'result_arr' is not defined

In [ ]:
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)
# print('y_train shape: ', y_train.shape)
# print('y_test shape: ', y_test.shape

# Build Neural Network

## Configurations

In [5]:
# Network parameters
n_hidden_1 = 256 # Units in first hidden layer
n_hidden_2 = 128 # Units in second hidden layer
n_input = 784 # Fashion MNIST data input (img shape: 28*28)
n_classes = 10 # Fashion MNIST total classes (0-9 digits)
image_size = 28
channel_size = 1
n_samples =  x_train.shape[0]
size, num_features = train_images.shape
batch_size = 50
training_epochs = 20
num_iterations = size//batch_size
test_step = 500

filter_size1 = 512
filter_size2 = 128
filter_size3 = 32

sigma = 1e-3
learning_rate = 0.01

LOG_DIR = 'projector'

NameError: name 'x_train' is not defined

In [ ]:
def variable_summaries(var):
    '''attach a lot of summaries to a tensorboard'''
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('seddev',stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram',var)

## Input 

In [64]:
if FLAGS.job_name == 'ps':
    server.join()
elif FLAGS.job_name == 'worker':
    load_data()
    
    # Assign operations to local server
    with tf.device(tf.train.replica_device_setter(
            worker_device="/job:worker/task:%d" % FLAGS.task_index,
            cluster=cluster)):
        keras.backend.set_learning_phase(1)
        keras.backend.manual_variable_initialization(True)
        model = create_model()
        
        
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None,image_size, image_size,channel_size])
        targets = tf.placeholder(tf.float32, shape=[None, 100], name="y-input")
        tf.summary.image('input',x,10)
        
    with tf.name_scope('transfer_learning):
        img_input = Input(shape = (256,256,3))
        model = ResNet50(input_tensor = img_input, weights ='imagenet',include_top = True)
        model.summary()
        for layer in model.layers[:]:
            layer.trainable=False

        x = model.output
        y_pred = Dense(100,activations = 'softmax')(x)

        model_final = Model(input = model.input, output= y_pred)
                       
                       
     with name_scope('train_step'):
        preductions = model_final.output
        loss = tf.reduce_mean(keras.losses.categorical_crossentropy(targets, predictions))
        tf.summary.scalar('loss',cost)
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    summary_op = tf.summary.merge_all()

    global_step = tf.get_variable('global_step', [],
                                      initializer=tf.constant_initializer(0),
                                      trainable=False)
        init_op = tf.global_variables_initializer()

    sv = tf.train.Supervisor(is_chief=(FLAGS.task_index == 0),
                             global_step=global_step,
                             logdir="train_logs",
                             save_model_secs=600,
                             init_op=init_op)

    print("Waiting for other servers")

    start_time = time.time()
     with sv.managed_session(server.target) as sess:
            keras.backend.set_session(sess)
            step = 0
        for epoch in range(training_epochs):
            batchs = int(n_samples/batch_size)
            count = 0
            for i in range(batchs):
                offset = (i*batch_size) % n_samples
                batch_x = x_train[offset:(offset+batch_size),:]
                batch_y = y_train[offset:(offset+batch_size),:]

                summary, _,loss,step = sess.run([merged, optimizer,cross_entropy,global_step],feed_dict={x: batch_x, y: batch_y})
                train_writer.add_summary(summary,i)

                count += 1
                if count % frequency == 0 or i+1 == batch_count:
                    elapsed_time = time.time() - start_time 
                    start_time = time.time()
                    print('Step: %d,'%(step + 1),
                         "Epoch: %2d, "%(epoch+1),
                         'Cost: %.4f,'%loss,
                         'AvgTime: %3.2fms'%float(elapsed_time*100/frequency))

        for i in range(test_step):
            if i % 10 == 0:
                summary, test_accuracy = sess.run([merged,accuracy],feed_dict={x: x_test, y: y_test})
                test_writer.add_summary(summary, i)
                print('Test accuracy at step %s: %s' % (i, test_accuracy)) 
    sv.stop()
    print("done")

## Define Model 

In [68]:
img_input = Input(shape = (256,256,3))
model = ResNet50(input_tensor = img_input, weights ='imagenet',include_top = True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_11 (ZeroPadding2 (None, 262, 262, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        zero_padding2d_11[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation